In [ ]:
from pickle_file import load_obj,save_obj
from tqdm import tqdm
import os
from datetime import datetime
import numpy as np
import networkx as nx
import igraph as ig
from ONet_fd import ONet_fd
import random as rd
import pandas as pd
import copy

save_time = '2020may'


## fd_set 文件位置：'obj_fd_set/fd_set_' + fd
## o_s_vector 文件位置： 'obj_osv_n/osv_n_' + fd, skill的顺序文件是 'obj_o_s_vector/skill_list'
fd_list = load_obj('folder_list')
#fd_increase1 = load_obj('fd_increase1')
fd_increase2 = load_obj('fd_increase2')
skill_list_std = load_obj('obj_osv_n/skill_list')
skill_dict = load_obj('skill_dict')
fd_time = load_obj('fd_time')
occupation_name = load_obj('occupation_name_increase2')
socio_skills = load_obj('social_cognitive_skills')
physical_skills = load_obj('sensory_physical_skills')

In [ ]:
for fd in tqdm(fd_increase2):
    fd_osv_n = load_obj('osv_n_6digit/osv_n_6digit_'+fd)
    occupations_6digit = load_obj('osv_n_6digit/occupations_6digit_'+fd)
    skillsim = {}
    
    for o1 in occupations_6digit:
        skillsim[o1] = {}
        for o2 in occupations_6digit:
            if o1 != o2:
                temp1 = 0
                temp2 = 0
                for i in range(0,len(skill_list_std)):
                    temp1 += min([fd_osv_n[o1][i], fd_osv_n[o2][i]])
                    temp2 += max([fd_osv_n[o1][i], fd_osv_n[o2][i]])
                    
                skillsim[o1][o2] = temp1/temp2
            
    save_obj(skillsim, 'skillsim_6digit/skillsim_'+fd)

In [ ]:
def get_jobs_tuplelist_skillsim(skillsims, job_list, TS = 0.0):
    sm_tuple = []
    for i in range(0,len(job_list)):
        for j in range(0,len(job_list)):
            if (i > j):
                j1 = job_list[i]
                j2 = job_list[j]
                sm = skillsims[j1][j2]
                if sm > TS :
                    sm_tuple.append((j1,j2,sm))
            
    return sm_tuple
    
    

for fd in tqdm(fd_increase2):
    skillsim = load_obj('skillsim_6digit/skillsim_' + fd)
    occupations_6digit = load_obj('osv_n_6digit/occupations_6digit_' + fd)
    G = ig.Graph.TupleList(get_jobs_tuplelist_skillsim(skillsim,occupations_6digit),
                                        edge_attrs = 'weight')
    save_obj(G,'skillsim_6digit_graph/skillsim_graph_6digit_' + fd)

In [ ]:
from influencial_spreader_node import skillsim_weighted_ci

fd = fd_increase2[-1]
thre = 0
ci_order = 1

ci_nodes_city_skillsim, isolated_nodes_city_skillsim, G_in = skillsim_weighted_ci(thre, fd, ci_order)


def emp_ci(ci_city):

    emp_ci_city = {}
    
    save_time = '2020may'
    city_codes = load_obj('occupations_city/' + save_time +'/city_codes_'+ save_time)
    city_names = load_obj('occupations_city/' + save_time +'/city_names_'+ save_time)
    city_occupations = load_obj('occupations_city/' + save_time +'/city_occupations_'+ save_time)
    city_occupations_names = load_obj('occupations_city/' + save_time +'/city_occupations_names_'+ save_time)
    city_emp = load_obj('occupations_city/' + save_time +'/city_emp_'+ save_time)

    for city in city_codes:
        temp_lbci = []
        temp_names = []

        #total_emp = sum(city_emp[city])
        for i in range(0, len(ci_city[city][1])):
            o = ci_city[city][1][i]
            ci = ci_city[city][0][i]
            temp_lbci.append(np.log(city_emp[city][o]) * ci)
            temp_names.append(o)


        weighted_ci_values, weighted_ci_names = zip(*sorted(zip(temp_lbci,temp_names), reverse = True))

        emp_ci_city[city] = [weighted_ci_values, weighted_ci_names]

    return emp_ci_city


def emp_ci_normalized(ci_city):

    emp_ci_city = {}
    
    save_time = '2020may'
    city_codes = load_obj('occupations_city/' + save_time +'/city_codes_'+ save_time)
    city_names = load_obj('occupations_city/' + save_time +'/city_names_'+ save_time)
    city_occupations = load_obj('occupations_city/' + save_time +'/city_occupations_'+ save_time)
    city_occupations_names = load_obj('occupations_city/' + save_time +'/city_occupations_names_'+ save_time)
    city_emp = load_obj('occupations_city/' + save_time +'/city_emp_'+ save_time)

    for city in city_codes:
        temp_emp = []
        temp_ci = []
        temp_names = []

        #total_emp = sum(city_emp[city])
        for i in range(0, len(ci_city[city][1])):
            o = ci_city[city][1][i]
            if (o in city_occupations[city]) and (city_emp[city][o] != 0):
                temp_ci.append(ci_city[city][0][i])
                temp_emp.append(np.log(city_emp[city][o]))
                temp_names.append(o)
                               
        temp_lbci = []
        temp_names_lbci = []
        min_ci = min(temp_ci)
        max_ci = max(temp_ci)
        min_emp = min(temp_emp)
        max_emp = max(temp_emp)
        
        for ci, emp, name in zip(temp_ci, temp_emp, temp_names):
            temp_lbci.append(ci * emp)
            temp_names_lbci.append(name)

        weighted_ci_values, weighted_ci_names = zip(*sorted(zip(temp_lbci,temp_names_lbci), reverse = True))

        emp_ci_city[city] = [weighted_ci_values, weighted_ci_names]

    return emp_ci_city


skillsim_emp_ci = emp_ci(ci_nodes_city_skillsim)